# Setup system

### Imports

In [ ]:
import FISH2_functions

### Find machine address
Use the below function to find the USB address and serial number of the connected devices, one device at a time.  
If you know the identifier of the FTDI chip of your device use it as input.  
Otherwise follow the instructions of the function and unplug and plug your device to find the address.  
  
When the search is successful add the serial number (preferred) or the USB COM port (alternative) to the data file.  
Open the data file with the `ROBOFISH_user_program.py` and add the info to the `ROBOFISH_System_datafile.yaml`.

In [ ]:
FISH2_functions.find_address(identifier=None)

# Initiate system

In [ ]:
import FISH2_functions

#System specific paths to database that is used to keep track of the experiment data. 
#database is automatically generated by the "FISH2_user_program.py" and prints the path after runing, copy the path to this location 
db_path = 'FISH_database\FISH_System2_db.sqlite'

#System specific path to start_imaging_file
start_imaging_file_path = "C:\\Users\\BL\\Desktop\\ROBOFISH_next\\FISH_database\\start_imaging_file.txt"

#System specific path to where the microscope saves the images. The info files will be put here.
#Use double slashes "\\" but do not put the trailing slashes! Example "C:\\Folder\\subfolder"
imaging_output_folder = "G:\\To_Monod\\EXP_test"


F2 = FISH2_functions.FISH2(db_path, imaging_output_folder, start_imaging_file_path, system_name='ROBOFISH2')

# Protocol steps

In [ ]:
def SDSclearing(chamber, volume, cycles, incubation_time=0):
    """
    SDS clearing of tissue. Including SSC2X wash afterwards.
    Input:
    `chamber`(str): Chamber to clear. Like: 'Chamber1'.
    `volume`(int): Volume of SDS to use per wash round.
    `cycles`(int): Number of cycles to wash.
    `incubation_time`(int): Incubation time in minutes per wash cycle.
    
    """
    F2.L.logger.info('SDS clearing {} start'.format(chamber))
    F2.setTemp(20, chamber) #Set to room temperature
    #Wash with SDS
    F2.extractDispenseBuffer('SDS', volume, chamber, padding=True, same_buffer_padding=True, double_volume=True, m=incubation_time)
    F2.resetReservoir(50, update_buffer=True)
    for i in range(cycles-1):
        F2.extractDispenseBuffer('SDS', volume, chamber, padding=False, m=incubation_time)
        F2.resetReservoir(50, update_buffer=True)
    #Wash out SDS with SSC2X
    for i in range(5):
        F2.extractDispenseSSC(1000, chamber)
        time.sleep(60)
    F2.L.logger.info('Clearing {}. Washed {} times {} minutes with {}ul SDS.'. format(chamber, cycles, incubation_time, volume))
        
def heatShock(chamber, TE_volume, staining_temp, heatshock_temp, incubation_time=0):
    """
    Heatshock of tissue in TE buffer. Including SSC2X wash afterwards.
    Input:
    `chamber`(str): Chamber to do heatshock. Like: 'Chamber1'.
    `TE_volume`(int): Volume of TE to use.
    `staining_temp`(int): Temperature used for hybridization/washes in Celcius.
    `heatshock_temp`(int): Temperature for heatshock in Celcius. 
    `incubation_time`(int): Time to keep the tissue at the heatschok_temp in minutes.
    
    """
    F2.L.logger.info('Heatshock {} start'.format(chamber))
    F2.extractDispenseBuffer('TE', TE_volume, chamber, padding=True, same_buffer_padding=True)
    time.sleep(60)
    F2.extractDispenseBuffer('TE', TE_volume, chamber, padding=False)
    F2.resetReservoir(50, update_buffer=True)
    F2.setRampTemp(heatshock_temp, step=1, step_time=2)
    F2.waitTemp(heatshock_temp, chamber, error=5, sd=0.01, verbose=True)
    time.sleep(60 * incubation_time)
    F2.setRampTemp(staining_temp, chamber, step=1, step_time=2)
    time.sleep(30)
    for i in range(2):
        F2.extractDispenseSSC(1000, chamber)
        time.sleep(60)
    F2.waitTemp(staining_temp, chamber, error=3, sd=0.01, verbose=False)
    F2.L.logger.info('Heatshock {}. Washed with {} TE buffer and perfomed heatshock for {} minutes at {}C.'.format(chamber, TE_volume, incubation_time, heatshock_temp))
    
def hybridization(chamber, cycle, hybridization_temp, indirect=None, incubation_time=0):
    """
    Hybridization.
    Input:
    `chamber`(str): Chamber to hybridize. Like: 'Chamber1'.
    `cycle`(int): Current staining cycle.
    `hybridization_temp`(float): Hybridization temperature.
    `indirect`(str): "A" for encoding probes, "B" for detection probes. None
        for an experiment with direct labeling.
    `incubation_time`(int): Hybridization time in HOURS
    
    """
    F2.setTemp(hybridization_temp, chamber)
    F2.waitTemp(hybridization_temp, chamber, error=3, sd=0.01, verbose=False)
    F2.L.logger.info('Hybridization {}, cycle {}, indirect {}, start dispense'.format(chamber, cycle, indirect))
    F2.extractDispenseHybmix(chamber, cycle, indirect=indirect, h=incubation_time)
    F2.L.logger.info('Hybridization {}, cycle {}, indirect {}, finished'.format(chamber, cycle, indirect))
    
def washBufferWash(chamber, volume, cycles, incubation_time=0):
    """
    Stringency wash after hybridization. Including SSC2X wash afterwards.
    Input:
    `chamber`(str): Chamber to wash. Like: 'Chamber1'.
    `volume`(int): Volume of WB to use per wash cycle.
    `cycles`(int): Number of cycle to wash.
    `incubation_time`(int): Incubation time in minutes per wash cycle.
    
    """
    F2.L.logger.info('Stingency wash {} start'.format(chamber))
    #Wash with WB
    F2.extractDispenseBuffer('WB', volume, chamber, padding=True, same_buffer_padding=True, double_volume=True, m=incubation_time)
    F2.resetReservoir(50, update_buffer=True)
    for i in range(cycles-1):
        F2.extractDispenseBuffer('WB', volume, chamber, padding=False, m=incubation_time)
        F2.resetReservoir(50, update_buffer=True)
    #Wash out WB
    for i in range(4):
        F2.extractDispenseSSC(1000, chamber)
    F2.L.logger.info('Stringency wash {}. Washed {} times with {}ul WB for {} minutes'.format(chamber, cycles, volume, incubation_time))    

def imagingBuffer(chamber, volume, imaging_temp):
    """
    Inject imaging buffer into the chamber and set chamber to imaging temperature.
    Input:
    `chamber`(str): Chamber to inject. Like: 'Chamber1'.
    `volume`(int): Volume of imaging buffer to inject.
    `imaging_temp`(float): Temperature to perform the imaging at.
    
    """
    F2.L.logger.info('Imaging buffer injecting into {}'.format(chamber))
    F2.setTemp(imaging_temp, chamber)
    F2.extractDispenseBuffer('IB', volume, chamber, padding=True, same_buffer_padding=False)
    F2.waitTemp(imaging_temp, chamber, error=3, sd=0.01, verbose=False)
    F2.L.logger.info('Imaging buffer. Injected {}ul of imaging buffer into {}, temperature set to {}C'.format(volume, chamber, imaging_temp))
    
def stripping(chamber, volume, cycles, stripping_temp, incubation_time=0, wash_cycles= 5):
    """
    Stripping of hybridized probes. Including SSC2X wash before (to wash out IB)
    and after.
    Input:
    `chamber`(str): Chamber to wash. Like: 'Chamber1'.
    `volume`(int): Volume of SB to use per wash cycle.
    `cycles`(int): Number of cycles to wash.
    `incubation_time`(int): Incubation time in minutes per wash cycle.
    `stripping_temp`(int): Temperature used for hybridization/washes in Celcius.
    `wash_cycles`(int): Number of 1000ul washes after the striping. Default=5
    
    """
    F2.L.logger.info('Stripping {} start'.format(chamber))
    #Set temperature back to stripping_temperature
    F2.setTemp(stripping_temp, chamber)
    #Wash out imaging buffer
    for i in range(4):
        F2.extractDispenseSSC(1000, chamber)
    F2.waitTemp(stripping_temp, chamber, error=3, sd=0.01, verbose=False)
    F2.extractDispenseBuffer('SB', volume, chamber, padding=True, same_buffer_padding=True, double_volume=True, m=incubation_time)
    F2.resetReservoir(50, update_buffer=True)
    for i in range(cycles-1):
        F2.extractDispenseBuffer('SB', volume, chamber, padding=False, m=incubation_time)
        F2.resetReservoir(50, update_buffer=True)   
    #Wash out SB
    for i in range(wash_cycles):
        F2.extractDispenseSSC(1000, chamber)
    F2.L.logger.info('Stripping. Washed {}, {} times with {}ul SB for {} minutes'.format(chamber, cycles, volume, incubation_time))

# Combined functions

In [ ]:
def osmFISH_first_part(chamber, cycle):
    """
    Perform first part of protocol.
    SDS-clearing, heatshock, hybridization, stringency wash, start imaging.
    Using a direct labeling approach.
    Input:
    `chamber`(str): Chamber to wash. Like: 'Chamber1'.
    `cycle`(int): Current staining cycle of experiment.
    
    """
    volume = F2.Parameters['Hybmix_volume']
    
    #SDS clearing
    SDSclearing(chamber, volume, 4, 5)
    
    #Skipping heatshock of original osmFISH protocol. 
    
    #Hybridization
    hyb_time_code = 'Hyb_time_{}_A'.format(chamber[-1])
    hyb_time = F2.Parameters[hyb_time_code]
    hybridization(chamber, cycle, F2.Parameters['Staining_temperature'], indirect=None, incubation_time=hyb_time)
    
    #Wash buffer wash
    washBufferWash(chamber, 750, 4, 15)

    #Inject imaging buffer
    imagingBuffer(chamber, volume, F2.Parameters['Imaging_temperature'])
    
    perif.sent_push(short_message='Ready for Imaging')

def osmFISH_repeat_part(chamber, cycle):
    """
    Perform second part of protocol.
    Stripping, hybridization, stringency wash, start imaging.
    Using a direct labeling approach.
    Input:
    `chamber`(str): Chamber to wash. Like: 'Chamber1'.
    `cycle`(int): Current cycle of experiment.
    `volume`(int): Volume to use for all washes. Should be chamber volume.
    `hyb_time`(int): Hybridization time in HOURS
    
    """
    volume = F2.Parameters['Hybmix_volume']

    #Stripping
    stripping(chamber, 750, 3, F2.Parameters['Stripping_temperature'], 10)
    F2.setTemp(F2.Parameters['Staining_temperature'], chamber)
    
    #perif.sent_push(short_message='Done with stripping check results')
    #input('Press enter to continue...')

    #Hybridization
    hyb_time_code = 'Hyb_time_{}_A'.format(chamber[-1])
    hyb_time = F2.Parameters[hyb_time_code]
    hybridization(chamber, cycle, F2.Parameters['Staining_temperature'], indirect=None, incubation_time=hyb_time)

    #Wash buffer wash
    washBufferWash(chamber, 750, 4, 15)

    #Inject imaging buffer
    imagingBuffer(chamber, volume, F2.Parameters['Imaging_temperature'])

    perif.sent_push(short_message='{} Ready for Imaging, Cycle: {}'.format(chamber, cycle))

# Scheduler

In [ ]:
F2.scheduler(osmFISH_first_part, osmFISH_repeat_part, remove_experiment=False, log_info_file=True)